In [30]:
from astropy.coordinates import SkyCoord, get_sun, get_moon, EarthLocation, AltAz
from astropy.time import Time
from astropy.table import Table, Column
import astropy.units as u
import numpy as np
from astropy import constants as c
from fils import *
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.optimize import root
from matplotlib.dates import DateFormatter, num2date
from astroplan.moon import moon_illumination
from astroplan.plots import plot_finder_image
import datetime
from astroplan import download_IERS_A
from time import sleep
from tqdm import tqdm_notebook, tnrange

In [31]:
def obshelp(objs, date, observatory='skinakas', utcdiff=(0.0*u.h), 
            civang=(6.0*u.deg), amlim=2.0, astrang=(18.0*u.deg), moon=True, 
            FC=False, fov=30.0, retic=True, showtabs=False, addobstim=None, 
            mkalmanac=True, mkamplot=True, enforce_iers=False, prtalm=True):
    if enforce_iers:
        while True:
            try:
                download_IERS_A()
                break
            except:
                pass
    else:
        try:
            download_IERS_A()
        except:
            sleep(1)
            try:
                download_IERS_A()
            except:
                sleep(1)
                try:
                    download_IERS_A()
                except:
                    print('Outdated Earth Location Data')
                    pass
    
    master = objs
    
    deglim = 90.0-np.rad2deg(np.arccos(1.0/amlim))

    timediff = 10.0
    civang = float(civang.to(u.deg).value)
    amlim = float(amlim)
    astrang = float(astrang.to(u.deg).value)
    
    if isinstance(master['RA'][0], float) or isinstance(master['RA'][0], int):
        coos = SkyCoord(ra=master['RA'], dec=master['Dec'], frame='icrs', unit=(u.deg, u.deg))
    else:
        coos = SkyCoord(ra=master['RA'], dec=master['Dec'], frame='icrs', unit=(u.hourangle, u.deg))

    coocol = Column(coos, name='Coordinate')
    master.add_column(coocol)

    Fv = np.multiply(np.multiply(3631.0, np.power(10.0, np.negative(np.divide(
        np.array(master['Mag']).astype(float), 2.5)))), np.power(10.0, -23.0))
    FvCol = Column(Fv, name='Fv')
    master.add_column(FvCol)

    Fl = []
    for i in range(0, len(master)):
        Fl.append(np.multiply(np.divide((c.c.to(u.angstrom/u.s)).value,
                                        np.power(effwav(master['Filter'][i]), 2.0)), master['Fv'][i]))
    FlCol = Column(Fl, name='Fl')
    master.add_column(FlCol)

    if showtabs:
        master.show_in_notebook()

    obslat, obslon, obsheight = observatoryephem(observatory)
    observatoryloc = EarthLocation(
        lat=obslat*u.deg, lon=obslon*u.deg, height=obsheight*u.m)

    if date == 'today':
        date = datetime.datetime.utcnow()
        date = date.strftime('%Y-%m-%d')

    probedate = Time(date+'T23:59:59', format='isot', scale='utc')
    probedate = probedate+(np.linspace(-timediff, timediff, 1000)*u.hour)
    probesunaltaz = get_sun(probedate).transform_to(
        AltAz(obstime=probedate, location=observatoryloc))

    probearray = probesunaltaz.alt.degree

    probeinterp = interp1d(range(0, len(probearray)), probearray)
    roots = root(probeinterp, [100, len(probearray)-100])
    nr = np.rint(roots.x).astype(int)
    ssind = nr[0]
    srind = nr[1]

    cvprobeint = interp1d(range(0, len(probearray)),
                          np.add(probearray, civang))
    roots = root(cvprobeint, [100, len(probearray)-100])
    nr = np.rint(roots.x).astype(int)
    cvtwind = nr[0]
    cvbdind = nr[1]

    astrprobeint = interp1d(range(0, len(probearray)),
                            np.add(probearray, astrang))
    roots = root(astrprobeint, [100, len(probearray)-100])
    nr = np.rint(roots.x).astype(int)
    astrtwind = nr[0]
    astrbdind = nr[1]

    tss = probedate[ssind]
    tct = probedate[cvtwind]
    tat = probedate[astrtwind]
    tad = probedate[astrbdind]
    tcd = probedate[cvbdind]
    tsr = probedate[srind]
    tmn = tss+((tsr-tss).to(u.h))/2.0
    if prtalm:
        print('Sunset:\t', tss)
        print('Civil Twilight:\t', tct)
        print('Astronomical Twilight:\t', tat)
        print('Midnight:\t', tmn)
        print('Astronomical Dawn:\t', tad)
        print('Civil Dawn:\t', tcd)
        print('Sunrise:\t', tsr)

    times = np.array(
        [((i/1000.0)*(tsr-tss)).to(u.h).value for i in range(-10, 1011)])*u.hour
    times = tss + times
    sunaltaz = get_sun(times).transform_to(
        AltAz(obstime=times, location=observatoryloc))
    moonaltaz = get_moon(times).transform_to(
        AltAz(obstime=times, location=observatoryloc))
    millum = int(np.rint(moon_illumination(tss+((tsr-tss).to(u.h))/2.0)*100))
    
    if mkamplot:
        LINE_STYLES = ['-', '--', '-.', ':']
        NUM_STYLES = len(LINE_STYLES)
        NUM_COLORS = np.ceil(len(master)/NUM_STYLES)
        plt.clf()
        fig = plt.figure(dpi=350, figsize=(16, 12), facecolor='white')
        ax = fig.add_subplot(111)
        cm = plt.get_cmap('tab20b')
        formatindex=1
        colorindex=1
        for pb in tqdm_notebook(range(len(master)), desc='Plotting'):
            i = master[pb]
            objaltaz = i['Coordinate'].transform_to(
                AltAz(obstime=times, location=observatoryloc))
            nameof = i['Name']
            ax.plot_date(times.plot_date, objaltaz.alt.deg, fmt=LINE_STYLES[formatindex-1], xdate=True, label=nameof, lw=4, color=cm(colorindex/NUM_COLORS), alpha=0.8)
            if formatindex==NUM_STYLES:
                formatindex=1
                colorindex+=1
            else:
                formatindex+=1
        if moon:
            ax.plot_date(times.plot_date, moonaltaz.alt.deg, fmt='--',
                         color='yellow', xdate=True, label='Moon')
        ax.xaxis.set_major_formatter(DateFormatter('%H:%M'))
        ax.set_xlabel('UTC')
        ax.set_ylabel('Altitude (deg)')
        ax.set_ylim(0, 90)
        ax.set_xlim(times[0].plot_date, times[-1].plot_date)
        ax.axvspan(times[0].plot_date, times[-1].plot_date,
                   color='darkblue', alpha=0.2, zorder=0)
        ax.axvspan(tct.plot_date, tcd.plot_date,
                   color='darkblue', alpha=0.5, zorder=1)
        ax.axvspan(tat.plot_date, tad.plot_date,
                   color='darkblue', alpha=0.9, zorder=2)

        def alt2am(alt):
            inlab = np.array(alt).astype(float)
            toret = np.around(np.divide(1.0, np.cos(
                np.deg2rad(np.subtract(90.0, inlab)))), decimals=2)
            toretn = [x if x < 100 else 'inf' for x in toret]
            return toretn

        ax2 = ax.twinx()
        ax2.set_ylim(ax.get_ylim())
        ax2.set_yticklabels(alt2am(ax.get_yticks()))
        ax2.set_ylabel('Airmass')

        limlin = 90.0-np.rad2deg(np.arccos(1.0/amlim))

        def loctim(tim):
            ot = num2date(tim)
            toadd = [datetime.timedelta(hours = utcdiff.to(u.hour).value) for i in ot]
            nt = [ot[i]+toadd[i] for i in range(0, len(ot))]
            nnt = [i.strftime('%H:%M') for i in nt]
            return nnt

        ax3 = ax.twiny()
        ax3.set_xlim(ax.get_xlim())
        ax3.set_xticks(ax.get_xticks())
        ax3.set_xticklabels(loctim(ax.get_xticks()))
    #     ax3.xaxis.set_major_formatter(DateFormatter('%H:%M'))
        ax3.set_xlabel('LT')

        ax.axhline(y=limlin, color='red', linestyle='-.')
    #     plt.gcf().autofmt_xdate()
        ax.legend(loc='best')
        ax.set_title(date + ' Moon at %d%%' % millum)
        plt.tight_layout()
        plt.savefig('RESULTS/AMPLOT.png')
        
    
    if addobstim!=None:
        if 'ast' in addobstim:
            obstimes = np.array([((i/1000.0)*(tad-tat)).to(u.h).value for i in range(0, 1001)])*u.hour
            obstimes = tat + obstimes
            objtimes = []
            for pb2 in tqdm_notebook(range(len(master)), desc='Obs. Time Ast. TW'):
                i = master[pb2]
                objalt = np.array(i['Coordinate'].transform_to(AltAz(obstime=obstimes, location=observatoryloc)).alt.deg)
                if not np.any(objalt>deglim):
                    objtimes.append((0.0*u.s).to(u.h))
                else:
                    if objalt[0]>deglim and objalt[-1]>deglim:
                        objr = obstimes[0]
                        objs = obstimes[-1]
                        objtimes.append((objs-objr))
                    elif objalt[0]>deglim and (not objalt[-1]>deglim):
                        objr = obstimes[0]
                        objprobe = interp1d(range(0, len(objalt)),np.subtract(objalt, deglim), fill_value='extrapolate')
                        roots = root(objprobe, [500])
                        nr = np.rint(roots.x).astype(int)
                        objs = obstimes[nr[0]]
                        objtimes.append((objs-objr))
                    elif (not objalt[0]>deglim) and (objalt[-1]>deglim):
                        objs = obstimes[-1]
                        objprobe = interp1d(range(0, len(objalt)),np.subtract(objalt, deglim), fill_value='extrapolate')
                        roots = root(objprobe, [500])
                        nr = np.rint(roots.x).astype(int)
                        objr = obstimes[nr[0]]
                        objtimes.append((objs-objr))
                    else:                         
                        objprobe = interp1d(range(0, len(objalt)),np.subtract(objalt, deglim), fill_value='extrapolate')
                        roots = root(objprobe, [100, len(objalt)-100])
                        nr = np.rint(roots.x).astype(int)
                        objr = obstimes[nr[0]]
                        objs = obstimes[nr[1]]
                        objtimes.append((objs-objr))
            objtimes = [j.value*24.0 for j in objtimes]
            astcol = Column(objtimes, name='TObsAstT')
            master.add_column(astcol)
            master['TObsAstT'].format='.2f'
            master['TObsAstT'].unit=u.h
        if 'civ' in addobstim:
            obstimes = np.array([((i/1000.0)*(tcd-tct)).to(u.h).value for i in range(0, 1001)])*u.hour
            obstimes = tct + obstimes
            objtimes = []
            for pb3 in tqdm_notebook(range(len(master)), desc='Obs. Time Civ. TW'):
                i = master[pb3]
                objalt = np.array(i['Coordinate'].transform_to(AltAz(obstime=obstimes, location=observatoryloc)).alt.deg)
                if not np.any(objalt>deglim):
                    objtimes.append((0.0*u.s).to(u.h))
                else:
                    if objalt[0]>deglim and objalt[-1]>deglim:
                        objr = obstimes[0]
                        objs = obstimes[-1]
                        objtimes.append((objs-objr))
                    elif objalt[0]>deglim and (not objalt[-1]>deglim):
                        objr = obstimes[0]
                        objprobe = interp1d(range(0, len(objalt)),np.subtract(objalt, deglim), fill_value='extrapolate')
                        roots = root(objprobe, [500])
                        nr = np.rint(roots.x).astype(int)
                        objs = obstimes[nr[0]]
                        objtimes.append((objs-objr))
                    elif (not objalt[0]>deglim) and (objalt[-1]>deglim):
                        objs = obstimes[-1]
                        objprobe = interp1d(range(0, len(objalt)),np.subtract(objalt, deglim), fill_value='extrapolate')
                        roots = root(objprobe, [500])
                        nr = np.rint(roots.x).astype(int)
                        objr = obstimes[nr[0]]
                        objtimes.append((objs-objr))
                    else:                         
                        objprobe = interp1d(range(0, len(objalt)),np.subtract(objalt, deglim), fill_value='extrapolate')
                        roots = root(objprobe, [100, len(objalt)-100])
                        nr = np.rint(roots.x).astype(int)
                        objr = obstimes[nr[0]]
                        objs = obstimes[nr[1]]
                        objtimes.append((objs-objr))
            objtimes = [j.value*24.0 for j in objtimes]
            civcol = Column(objtimes, name='TObsCivT')
            master.add_column(civcol)
            master['TObsCivT'].format='.2f'
            master['TObsCivT'].unit=u.h
        if 'ssr' in addobstim:
            obstimes = np.array([((i/1000.0)*(tsr-tss)).to(u.h).value for i in range(0, 1001)])*u.hour
            obstimes = tss + obstimes
            objtimes = []
            for pb4 in tqdm_notebook(range(len(master)), desc='Obs. Time SSR. TW'):
                i = master[pb4]
                objalt = np.array(i['Coordinate'].transform_to(AltAz(obstime=obstimes, location=observatoryloc)).alt.deg)
                if not np.any(objalt>deglim):
                    objtimes.append((0.0*u.s).to(u.h))
                else:
                    if objalt[0]>deglim and objalt[-1]>deglim:
                        objr = obstimes[0]
                        objs = obstimes[-1]
                        objtimes.append((objs-objr))
                    elif objalt[0]>deglim and (not objalt[-1]>deglim):
                        objr = obstimes[0]
                        objprobe = interp1d(range(0, len(objalt)),np.subtract(objalt, deglim), fill_value='extrapolate')
                        roots = root(objprobe, [500])
                        nr = np.rint(roots.x).astype(int)
                        objs = obstimes[nr[0]]
                        objtimes.append((objs-objr))
                    elif (not objalt[0]>deglim) and (objalt[-1]>deglim):
                        objs = obstimes[-1]
                        objprobe = interp1d(range(0, len(objalt)),np.subtract(objalt, deglim), fill_value='extrapolate')
                        roots = root(objprobe, [500])
                        nr = np.rint(roots.x).astype(int)
                        objr = obstimes[nr[0]]
                        objtimes.append((objs-objr))
                    else:                         
                        objprobe = interp1d(range(0, len(objalt)),np.subtract(objalt, deglim), fill_value='extrapolate')
                        roots = root(objprobe, [100, len(objalt)-100])
                        nr = np.rint(roots.x).astype(int)
                        objr = obstimes[nr[0]]
                        objs = obstimes[nr[1]]
                        objtimes.append((objs-objr))
            objtimes = [j.value*24.0 for j in objtimes]
            sscol = Column(objtimes, name='TObs')
            master.add_column(sscol)
            master['TObs'].format='.2f'
            master['TObs'].unit=u.h
        elif (('ssr'not in addobstim)and('ast'not in addobstim))and('civ'not in addobstim):
            print('Bad choice of observable time.')

    
    if mkalmanac:
        filer = open('RESULTS/ALMANAC.txt', 'w')
        filer.write('Sunset:\t\t\t\t\t' + tss.iso[:-7] + '\n')
        filer.write('Civil Twilight:\t\t\t' + tct.iso[:-7] + '\n')
        filer.write('Astronomical Twilight:\t' + tat.iso[:-7] + '\n')
        filer.write('Midnight:\t\t\t\t' + tmn.iso[:-7] + '\n')
        filer.write('Astronomical Dawn:\t\t' + tad.iso[:-7] + '\n')
        filer.write('Civil Dawn:\t\t\t\t' + tcd.iso[:-7] + '\n')
        filer.write('Sunrise:\t\t\t\t' + tsr.iso[:-7] + '\n')
        filer.close()
        
    if FC:
        if fov:
            try:
                fov=float(fov)*u.arcmin
            except ValueError:
                print('Bad FOV, setting to 0.5deg')
                fov=30.*u.arcmin
        else:
            fov=30.0*u.arcmin
        for fcpb in tqdm_notebook(range(len(master)), desc='FCs'):
            i = master[fcpb]
            plt.clf()
            plt.figure(dpi=350, figsize=(16, 12), facecolor='white')
            ax, hdu = plot_finder_image(i['Coordinate'], reticle=retic, fov_radius=fov)
            plt.tight_layout()
            plt.savefig('RESULTS/FC/'+i['Name']+'.png')
            
    master['Fv'].format='.2e'
    master['Fl'].format='.2e'
    
    master['Mag'].unit='AB'
    
    master['Fl'].unit=(u.erg/u.s/(u.cm*u.cm)/u.Hz)
    master['Fv'].unit=(u.erg/u.s/(u.cm*u.cm)/u.AA)
    
    master.remove_column('Coordinate')

    return master

In [32]:
# testl = Table.read('testinput.txt', format='ascii.csv')

In [33]:
# testl.show_in_notebook()

In [34]:
# testl = obshelp(objs=testl, date='2019-09-30', utcdiff=(3.0*u.hour), amlim=1.414213562373095, addobstim='astcivssr')

In [35]:
# testl = obshelp(objs=testl, date='2019-09-30', utcdiff=(3.0*u.hour), amlim=2.0, addobstim='ast', FC=True, fov=10., prtalm=False)

In [36]:
# testl.show_in_notebook()